In [1]:
import torch
from torch.utils.data import Dataset
import numpy as np
from torchvision import transforms
from datasets import Dataset as HFDataset, load_dataset, DatasetDict
from typing import Optional, Tuple, Dict, Any
from medmnist import BreastMNIST, PneumoniaMNIST
from transformers import AutoModelForImageClassification, AutoImageProcessor, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, recall_score, roc_auc_score
import random
from imblearn.under_sampling import NearMiss
import pandas as pd
from collections import Counter
from sklearn.utils import resample
import datasets



C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset Parsing

In [28]:
class MedMNISTtoHF(Dataset):
    def __init__(self, medmnist_dataset):
        """
        Convert MedMNIST dataset to a format compatible with HuggingFace models
        Args:
            medmnist_dataset: The original MedMNIST dataset
            transform: Optional transforms to be applied to the images
        """
        self.dataset = medmnist_dataset
        

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        img, label = self.dataset[idx]

        return {
            "image": img,
            "label": torch.tensor(label, dtype=torch.long)
        }

def convert_medmnist_to_hf(medmnist_dataset):
    """
    Convert a MedMNIST dataset to a HuggingFace dataset
    Args:
        medmnist_dataset: The original MedMNIST dataset
    Returns:
        The HuggingFace dataset
    """
    # Create wrapper dataset
    wrapper_dataset = MedMNISTtoHF(medmnist_dataset)
    
    # Convert to HF format
    images = []
    labels = []
    
    for i in range(len(wrapper_dataset)):
        sample = wrapper_dataset[i]
        images.append(sample["image"])
        labels.append(sample["label"].item())
    
    # Create HF dataset
    hf_dataset = HFDataset.from_dict({
        "image": images,
        "label": labels
    })
    
    return hf_dataset

def dataset_balancing(dataset, alpha):
    """	
    Balance the dataset by oversampling the minority class
    Args:
        dataset: The original dataset
        alpha: The oversampling factor
    Returns:
        The balanced dataset
    """

    mj_class = Counter(dataset['label']).most_common(1)[0][0]
    mn_class = abs(mj_class-1)
    data = dataset['image']

    mask = [lb== mj_class for lb in dataset['label']]
    X_majority = [img for img,flag in zip(data, mask) if flag]
    X_minority = [img for img,flag in zip(data, mask) if not flag]
    new_len_majority = len(X_minority) + int(alpha*len(X_minority))
    X_majority_resampled = resample(X_majority, 
                                    replace=False,  # No replacement
                                    n_samples=new_len_majority,  # Match minority class size
                                    random_state=42)
    X_resampled = X_majority_resampled + X_minority
    y_resampled = [mj_class]*new_len_majority + [mn_class]*len(X_minority)
    random.seed(42)
    random.shuffle(X_resampled)
    random.seed(42)
    random.shuffle(y_resampled)

    dict_blanced_dataset = {
        "image": X_resampled,
        "label": y_resampled
    }
    balanced_dataset = datasets.Dataset.from_dict(dict_blanced_dataset)
    return balanced_dataset

def load_dataset_medmnist(dataset_name, size, balancing):
    """
    Load a MedMNIST dataset and convert it to a HuggingFace dataset
    Args:
        dataset_name: The name of the MedMNIST dataset
        size: The size of the images
        balancing: Whether to balance the dataset
    Returns:
        The HuggingFace dataset
    """
    # Load MedMNIST dataset
    train_dataset = dataset_name(split='train', download=True, size=size)
    val_dataset = dataset_name(split='val', download=True, size=size)
    test_dataset = dataset_name(split='test', download=True, size=size)

    # Convert to HuggingFace dataset
    hf_train_dataset = convert_medmnist_to_hf(train_dataset)
    hf_val_dataset = convert_medmnist_to_hf(val_dataset)
    hf_test_dataset = convert_medmnist_to_hf(test_dataset)

    # Balancing
    if balancing:
        hf_train_dataset_balanced = dataset_balancing(hf_train_dataset, 0.5)
        dataset = DatasetDict({"train": hf_train_dataset_balanced, "validation": hf_val_dataset, "test": hf_test_dataset})
        return dataset
    else:
        dataset = DatasetDict({"train": hf_train_dataset, "validation": hf_val_dataset, "test": hf_test_dataset})
        return dataset

# Dataset Loading

In [29]:
breast_dataset_28 = load_dataset_medmnist(BreastMNIST, 28, False)
breast_dataset_balanced_28 = load_dataset_medmnist(BreastMNIST, 28, True)
breast_dataset_224 = load_dataset_medmnist(BreastMNIST, 224, False)
breast_dataset_balanced_224 = load_dataset_medmnist(BreastMNIST, 224, True)
# label = 1 => no disease

pneumonia_dataset_28 = load_dataset_medmnist(PneumoniaMNIST, 28, False)
pneumonia_dataset_balanced_28 = load_dataset_medmnist(PneumoniaMNIST, 28, True)
pneumonia_dataset_224 = load_dataset_medmnist(PneumoniaMNIST, 224, False)
pneumonia_dataset_balanced_224 = load_dataset_medmnist(PneumoniaMNIST, 224, True)
# label = 1 => with disease

Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\breastmnist_224.npz


100%|██████████| 4.17M/4.17M [00:00<00:00, 5.59MB/s]


Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist_224.npz
Using downloaded and verified file: C:\Users\baiet\.medmnist\pneumoniamnist_224.npz


# Freezer Functions

In [78]:
def beit_freezer(model):
    for name, param in model.named_parameters():
        if not name.startswith("classifier") \
            and not name.startswith("beit.pooler")\
            and not name.startswith("beit.encoder.layer.23")\
            and not name.startswith("beit.encoder.layer.22")\
            and not name.startswith("beit.encoder.layer.21")\
            and not name.startswith("beit.encoder.layer.20")\
            and not name.startswith("beit.encoder.layer.19"):
            param.requires_grad = False

def resnet_freezer(model):
    for name, param in model.named_parameters():
        if not name.startswith("classifier")\
            and not name.startswith("resnet.encoder.stages.3.layers.2"):
            param.requires_grad = False

# Reproducibility Function

In [18]:
def set_reproducibility(seed=42):
    # Set seeds for reproducibility
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Turn off to ensure deterministic results

# Model Function

In [83]:
def T_and_T(dataset, model_name, output_dir, batch_size, weight_decay, Training, Testing, freezer):
    # Set Seed
    set_reproducibility()

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Load the BEiT-large model and image processor
    model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True).to(device) 
    processor = AutoImageProcessor.from_pretrained(model_name)

    # Freeze all layers except the classifier and the last transformer layer
    freezer(model)

    # Verify which layers are trainable
    trainable_params = [name for name, param in model.named_parameters() if param.requires_grad]
    print(f"Trainable parameters: {trainable_params}")


    # Define preprocessing function
    def preprocess_images(examples):
            images = [processor(image.convert("RGB"), return_tensors="pt") for image in examples["image"]]
            pixel_values = torch.stack([image["pixel_values"].squeeze() for image in images])
            labels = torch.tensor(examples["label"])
            return {"pixel_values": pixel_values, "labels": labels}

    # Preprocess the dataset
    train_dataset = dataset["train"].with_transform(preprocess_images)
    validation_dataset = dataset["validation"].with_transform(preprocess_images)
    test_dataset = dataset["test"].with_transform(preprocess_images)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-4,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=5,
        weight_decay=weight_decay,
        logging_dir="./logs",
        logging_steps=10,
        save_total_limit=2,
        remove_unused_columns=False,
        push_to_hub=False,
        seed=42,
    )

    # Define Trainer
    trainer_beit = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        tokenizer=processor,
    )
    if Training:
        trainer_beit.train()
        model.save_pretrained(output_dir)
        processor.save_pretrained(output_dir)

    if Testing:
        predictions = trainer_beit.predict(test_dataset)
        pred_labels = predictions.predictions.argmax(axis=1)
        true_labels = predictions.label_ids

        # Calculate additional metrics
        accuracy = accuracy_score(true_labels, pred_labels)
        precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average="binary")
        specificity = recall_score(true_labels, pred_labels, pos_label=0)
        auc = roc_auc_score(true_labels, pred_labels)

        # Display the metrics
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision (weighted): {precision:.4f}")
        print(f"Recall (weighted): {recall:.4f}")
        print(f"F1-Score (weighted): {f1:.4f}")
        print(f"Specificity: {specificity:.4f}")
        print(f"AUC: {auc:.4f}")
        


# Training and Testing

In [45]:
T_and_T(breast_dataset_28, "microsoft/beit-large-patch16-224-pt22k", "beit_breast_28", 16, 0.01, True, True, beit_freezer)

Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-large-patch16-224-pt22k and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['beit.encoder.layer.19.lambda_1', 'beit.encoder.layer.19.lambda_2', 'beit.encoder.layer.19.attention.attention.query.weight', 'beit.encoder.layer.19.attention.attention.query.bias', 'beit.encoder.layer.19.attention.attention.key.weight', 'beit.encoder.layer.19.attention.attention.value.weight', 'beit.encoder.layer.19.attention.attention.value.bias', 'beit.encoder.layer.19.attention.output.dense.weight', 'beit.encoder.layer.19.attention.output.dense.bias', 'beit.encoder.layer.19.intermediate.dense.weight', 'beit.encoder.layer.19.intermediate.dense.bias', 'beit.encoder.layer.19.output.dense.weight', 'beit.encoder.layer.19.output.dense.bias', 'beit.encoder.layer.19.layernorm_before.weight', 'beit.encoder.layer.19.layernorm_before.bias', 'beit.encoder.layer.19.layernorm_after.weight', 'beit.encoder.layer.19.layernorm_after.bias', 'beit.encoder.layer.20.lambda_1', 'beit.encoder.layer.20.lambda_2', 'beit.encoder.layer.20.attention.attention.query.weight', 'beit.encoder

C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.581400,0.532466
2,0.545700,0.373951
3,0.425400,0.309652
4,0.230200,0.303705
5,0.239300,0.264146


Accuracy: 0.8910
Precision (weighted): 0.9008
Recall (weighted): 0.9561
F1-Score (weighted): 0.9277
Specificity: 0.7143
AUC: 0.8352


In [43]:
T_and_T(breast_dataset_balanced_28, "microsoft/beit-large-patch16-224-pt22k", "beit_breast_balanced_28", 16, 0, True, True, beit_freezer)

Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-large-patch16-224-pt22k and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['beit.encoder.layer.19.lambda_1', 'beit.encoder.layer.19.lambda_2', 'beit.encoder.layer.19.attention.attention.query.weight', 'beit.encoder.layer.19.attention.attention.query.bias', 'beit.encoder.layer.19.attention.attention.key.weight', 'beit.encoder.layer.19.attention.attention.value.weight', 'beit.encoder.layer.19.attention.attention.value.bias', 'beit.encoder.layer.19.attention.output.dense.weight', 'beit.encoder.layer.19.attention.output.dense.bias', 'beit.encoder.layer.19.intermediate.dense.weight', 'beit.encoder.layer.19.intermediate.dense.bias', 'beit.encoder.layer.19.output.dense.weight', 'beit.encoder.layer.19.output.dense.bias', 'beit.encoder.layer.19.layernorm_before.weight', 'beit.encoder.layer.19.layernorm_before.bias', 'beit.encoder.layer.19.layernorm_after.weight', 'beit.encoder.layer.19.layernorm_after.bias', 'beit.encoder.layer.20.lambda_1', 'beit.encoder.layer.20.lambda_2', 'beit.encoder.layer.20.attention.attention.query.weight', 'beit.encoder

C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.682800,0.711752
2,0.628900,0.442710
3,0.592400,0.359735
4,0.451900,0.278309
5,0.373300,0.277980


Accuracy: 0.8846
Precision (weighted): 0.9138
Recall (weighted): 0.9298
F1-Score (weighted): 0.9217
Specificity: 0.7619
AUC: 0.8459


In [46]:
T_and_T(breast_dataset_224, "microsoft/beit-large-patch16-224-pt22k", "beit_breast_224", 16, 0.1, True, True, beit_freezer)

Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-large-patch16-224-pt22k and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['beit.encoder.layer.19.lambda_1', 'beit.encoder.layer.19.lambda_2', 'beit.encoder.layer.19.attention.attention.query.weight', 'beit.encoder.layer.19.attention.attention.query.bias', 'beit.encoder.layer.19.attention.attention.key.weight', 'beit.encoder.layer.19.attention.attention.value.weight', 'beit.encoder.layer.19.attention.attention.value.bias', 'beit.encoder.layer.19.attention.output.dense.weight', 'beit.encoder.layer.19.attention.output.dense.bias', 'beit.encoder.layer.19.intermediate.dense.weight', 'beit.encoder.layer.19.intermediate.dense.bias', 'beit.encoder.layer.19.output.dense.weight', 'beit.encoder.layer.19.output.dense.bias', 'beit.encoder.layer.19.layernorm_before.weight', 'beit.encoder.layer.19.layernorm_before.bias', 'beit.encoder.layer.19.layernorm_after.weight', 'beit.encoder.layer.19.layernorm_after.bias', 'beit.encoder.layer.20.lambda_1', 'beit.encoder.layer.20.lambda_2', 'beit.encoder.layer.20.attention.attention.query.weight', 'beit.encoder

C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.581100,0.421221
2,0.501900,0.303186
3,0.403800,0.276382
4,0.122300,0.250387
5,0.155500,0.295957


Accuracy: 0.8782
Precision (weighted): 0.8740
Recall (weighted): 0.9737
F1-Score (weighted): 0.9212
Specificity: 0.6190
AUC: 0.7964


In [48]:
T_and_T(breast_dataset_balanced_224, "microsoft/beit-large-patch16-224-pt22k", "beit_breast_balanced_224", 16, 0.1, True, True, beit_freezer)

Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-large-patch16-224-pt22k and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['beit.encoder.layer.19.lambda_1', 'beit.encoder.layer.19.lambda_2', 'beit.encoder.layer.19.attention.attention.query.weight', 'beit.encoder.layer.19.attention.attention.query.bias', 'beit.encoder.layer.19.attention.attention.key.weight', 'beit.encoder.layer.19.attention.attention.value.weight', 'beit.encoder.layer.19.attention.attention.value.bias', 'beit.encoder.layer.19.attention.output.dense.weight', 'beit.encoder.layer.19.attention.output.dense.bias', 'beit.encoder.layer.19.intermediate.dense.weight', 'beit.encoder.layer.19.intermediate.dense.bias', 'beit.encoder.layer.19.output.dense.weight', 'beit.encoder.layer.19.output.dense.bias', 'beit.encoder.layer.19.layernorm_before.weight', 'beit.encoder.layer.19.layernorm_before.bias', 'beit.encoder.layer.19.layernorm_after.weight', 'beit.encoder.layer.19.layernorm_after.bias', 'beit.encoder.layer.20.lambda_1', 'beit.encoder.layer.20.lambda_2', 'beit.encoder.layer.20.attention.attention.query.weight', 'beit.encoder

C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.733000,0.631534
2,0.662500,0.700516
3,0.562800,0.300123
4,0.304500,0.285776
5,0.212900,0.257985


Accuracy: 0.8782
Precision (weighted): 0.8992
Recall (weighted): 0.9386
F1-Score (weighted): 0.9185
Specificity: 0.7143
AUC: 0.8264


In [ ]:
T_and_T(pneumonia_dataset_28, "microsoft/beit-large-patch16-224-pt22k", "beit_pneumonia_28", 16, 0.1, True, True, beit_freezer)

Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-large-patch16-224-pt22k and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['beit.encoder.layer.19.lambda_1', 'beit.encoder.layer.19.lambda_2', 'beit.encoder.layer.19.attention.attention.query.weight', 'beit.encoder.layer.19.attention.attention.query.bias', 'beit.encoder.layer.19.attention.attention.key.weight', 'beit.encoder.layer.19.attention.attention.value.weight', 'beit.encoder.layer.19.attention.attention.value.bias', 'beit.encoder.layer.19.attention.output.dense.weight', 'beit.encoder.layer.19.attention.output.dense.bias', 'beit.encoder.layer.19.intermediate.dense.weight', 'beit.encoder.layer.19.intermediate.dense.bias', 'beit.encoder.layer.19.output.dense.weight', 'beit.encoder.layer.19.output.dense.bias', 'beit.encoder.layer.19.layernorm_before.weight', 'beit.encoder.layer.19.layernorm_before.bias', 'beit.encoder.layer.19.layernorm_after.weight', 'beit.encoder.layer.19.layernorm_after.bias', 'beit.encoder.layer.20.lambda_1', 'beit.encoder.layer.20.lambda_2', 'beit.encoder.layer.20.attention.attention.query.weight', 'beit.encoder

C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.188500,0.259616
2,0.086100,0.080435
3,0.070400,0.112633
4,0.003100,0.089487
5,0.028100,0.085175


Accuracy: 0.8878
Precision (weighted): 0.8524
Recall (weighted): 0.9923
F1-Score (weighted): 0.9171
Specificity: 0.7137
AUC: 0.8530


In [50]:
T_and_T(pneumonia_dataset_balanced_28, "microsoft/beit-large-patch16-224-pt22k", "beit_pneumonia_balanced_28", 16, 0.1, True, True, beit_freezer)

Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-large-patch16-224-pt22k and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['beit.encoder.layer.19.lambda_1', 'beit.encoder.layer.19.lambda_2', 'beit.encoder.layer.19.attention.attention.query.weight', 'beit.encoder.layer.19.attention.attention.query.bias', 'beit.encoder.layer.19.attention.attention.key.weight', 'beit.encoder.layer.19.attention.attention.value.weight', 'beit.encoder.layer.19.attention.attention.value.bias', 'beit.encoder.layer.19.attention.output.dense.weight', 'beit.encoder.layer.19.attention.output.dense.bias', 'beit.encoder.layer.19.intermediate.dense.weight', 'beit.encoder.layer.19.intermediate.dense.bias', 'beit.encoder.layer.19.output.dense.weight', 'beit.encoder.layer.19.output.dense.bias', 'beit.encoder.layer.19.layernorm_before.weight', 'beit.encoder.layer.19.layernorm_before.bias', 'beit.encoder.layer.19.layernorm_after.weight', 'beit.encoder.layer.19.layernorm_after.bias', 'beit.encoder.layer.20.lambda_1', 'beit.encoder.layer.20.lambda_2', 'beit.encoder.layer.20.attention.attention.query.weight', 'beit.encoder

C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.121100,0.168538
2,0.118800,0.157667
3,0.113200,0.089077
4,0.013900,0.122651
5,0.042900,0.123828


Accuracy: 0.9103
Precision (weighted): 0.8884
Recall (weighted): 0.9795
F1-Score (weighted): 0.9317
Specificity: 0.7949
AUC: 0.8872


In [51]:
T_and_T(pneumonia_dataset_224, "microsoft/beit-large-patch16-224-pt22k", "beit_pneumonia_224", 16, 0.1, True, True, beit_freezer)

Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-large-patch16-224-pt22k and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['beit.encoder.layer.19.lambda_1', 'beit.encoder.layer.19.lambda_2', 'beit.encoder.layer.19.attention.attention.query.weight', 'beit.encoder.layer.19.attention.attention.query.bias', 'beit.encoder.layer.19.attention.attention.key.weight', 'beit.encoder.layer.19.attention.attention.value.weight', 'beit.encoder.layer.19.attention.attention.value.bias', 'beit.encoder.layer.19.attention.output.dense.weight', 'beit.encoder.layer.19.attention.output.dense.bias', 'beit.encoder.layer.19.intermediate.dense.weight', 'beit.encoder.layer.19.intermediate.dense.bias', 'beit.encoder.layer.19.output.dense.weight', 'beit.encoder.layer.19.output.dense.bias', 'beit.encoder.layer.19.layernorm_before.weight', 'beit.encoder.layer.19.layernorm_before.bias', 'beit.encoder.layer.19.layernorm_after.weight', 'beit.encoder.layer.19.layernorm_after.bias', 'beit.encoder.layer.20.lambda_1', 'beit.encoder.layer.20.lambda_2', 'beit.encoder.layer.20.attention.attention.query.weight', 'beit.encoder

C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.084500,0.082364
2,0.083100,0.086143
3,0.021500,0.082897
4,0.003200,0.050427
5,0.002700,0.057570


Accuracy: 0.8878
Precision (weighted): 0.8493
Recall (weighted): 0.9974
F1-Score (weighted): 0.9175
Specificity: 0.7051
AUC: 0.8513


In [52]:
T_and_T(pneumonia_dataset_balanced_224, "microsoft/beit-large-patch16-224-pt22k", "beit_pneumonia_balanced_224", 16, 0.1, True, True, beit_freezer)

Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-large-patch16-224-pt22k and are newly initialized: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['beit.encoder.layer.19.lambda_1', 'beit.encoder.layer.19.lambda_2', 'beit.encoder.layer.19.attention.attention.query.weight', 'beit.encoder.layer.19.attention.attention.query.bias', 'beit.encoder.layer.19.attention.attention.key.weight', 'beit.encoder.layer.19.attention.attention.value.weight', 'beit.encoder.layer.19.attention.attention.value.bias', 'beit.encoder.layer.19.attention.output.dense.weight', 'beit.encoder.layer.19.attention.output.dense.bias', 'beit.encoder.layer.19.intermediate.dense.weight', 'beit.encoder.layer.19.intermediate.dense.bias', 'beit.encoder.layer.19.output.dense.weight', 'beit.encoder.layer.19.output.dense.bias', 'beit.encoder.layer.19.layernorm_before.weight', 'beit.encoder.layer.19.layernorm_before.bias', 'beit.encoder.layer.19.layernorm_after.weight', 'beit.encoder.layer.19.layernorm_after.bias', 'beit.encoder.layer.20.lambda_1', 'beit.encoder.layer.20.lambda_2', 'beit.encoder.layer.20.attention.attention.query.weight', 'beit.encoder

C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.074700,0.059993
2,0.051000,0.136266
3,0.067600,0.093167
4,0.000800,0.067371
5,0.068900,0.070074


Accuracy: 0.9151
Precision (weighted): 0.8821
Recall (weighted): 0.9974
F1-Score (weighted): 0.9362
Specificity: 0.7778
AUC: 0.8876


In [ ]:
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-50", num_labels=2, ignore_mismatched_sizes=True)
for name, param in model.named_parameters():
    print(name)

In [85]:
T_and_T(breast_dataset_28, "microsoft/resnet-50", "resnet_breast_28", 16, 0, True, True, resnet_freezer)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['resnet.encoder.stages.3.layers.2.layer.0.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.1.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.2.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.bias', 'classifier.1.weight', 'classifier.1.bias']


C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.529100,0.620440
2,0.596400,0.555208
3,0.515500,0.534902
4,0.468900,0.522058
5,0.494700,0.516822


Accuracy: 0.7308
Precision (weighted): 0.7308
Recall (weighted): 1.0000
F1-Score (weighted): 0.8444
Specificity: 0.0000
AUC: 0.5000


In [84]:
T_and_T(breast_dataset_balanced_28, "microsoft/resnet-50", "resnet_breast_balanced_28", 16, 0, True, True, resnet_freezer)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['resnet.encoder.stages.3.layers.2.layer.0.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.1.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.2.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.bias', 'classifier.1.weight', 'classifier.1.bias']


C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.675000,0.675259
2,0.653000,0.658845
3,0.627600,0.575005
4,0.621700,0.569778
5,0.624300,0.552282


Accuracy: 0.7308
Precision (weighted): 0.7308
Recall (weighted): 1.0000
F1-Score (weighted): 0.8444
Specificity: 0.0000
AUC: 0.5000


In [86]:
T_and_T(breast_dataset_224, "microsoft/resnet-50", "resnet_breast_224", 16, 0, True, True, resnet_freezer)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['resnet.encoder.stages.3.layers.2.layer.0.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.1.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.2.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.bias', 'classifier.1.weight', 'classifier.1.bias']


C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.522800,0.616823
2,0.597100,0.558393
3,0.514900,0.543286
4,0.456100,0.532300
5,0.497200,0.530578


Accuracy: 0.7308
Precision (weighted): 0.7308
Recall (weighted): 1.0000
F1-Score (weighted): 0.8444
Specificity: 0.0000
AUC: 0.5000


In [87]:
T_and_T(breast_dataset_balanced_224, "microsoft/resnet-50", "resnet_breast_balanced_224", 16, 0, True, True, resnet_freezer)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['resnet.encoder.stages.3.layers.2.layer.0.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.1.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.2.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.bias', 'classifier.1.weight', 'classifier.1.bias']


C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.673500,0.652453
2,0.644100,0.629852
3,0.625800,0.584987
4,0.603200,0.572080
5,0.610300,0.559907


Accuracy: 0.7500
Precision (weighted): 0.7451
Recall (weighted): 1.0000
F1-Score (weighted): 0.8539
Specificity: 0.0714
AUC: 0.5357


In [88]:
T_and_T(pneumonia_dataset_28, "microsoft/resnet-50", "resnet_pneumonia_28", 16, 0, True, True, resnet_freezer)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['resnet.encoder.stages.3.layers.2.layer.0.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.1.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.2.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.bias', 'classifier.1.weight', 'classifier.1.bias']


C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.226300,0.217299
2,0.214700,0.186368
3,0.182300,0.168383
4,0.165800,0.168197
5,0.139800,0.163736


Accuracy: 0.8093
Precision (weighted): 0.7794
Recall (weighted): 0.9692
F1-Score (weighted): 0.8640
Specificity: 0.5427
AUC: 0.7560


In [89]:
T_and_T(pneumonia_dataset_balanced_28, "microsoft/resnet-50", "resnet_pneumonia_balanced_28", 16, 0, True, True, resnet_freezer)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['resnet.encoder.stages.3.layers.2.layer.0.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.1.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.2.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.bias', 'classifier.1.weight', 'classifier.1.bias']


C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.323900,0.252124
2,0.179100,0.217092
3,0.219700,0.195487
4,0.208000,0.216723
5,0.191600,0.200185


Accuracy: 0.8478
Precision (weighted): 0.8375
Recall (weighted): 0.9385
F1-Score (weighted): 0.8851
Specificity: 0.6966
AUC: 0.8175


In [90]:
T_and_T(pneumonia_dataset_224, "microsoft/resnet-50", "resnet_pneumonia_224", 16, 0, True, True, resnet_freezer)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['resnet.encoder.stages.3.layers.2.layer.0.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.1.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.2.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.bias', 'classifier.1.weight', 'classifier.1.bias']


C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.161400,0.178764
2,0.142000,0.152917
3,0.141200,0.136318
4,0.106900,0.123528
5,0.124100,0.124081


Accuracy: 0.8606
Precision (weighted): 0.8230
Recall (weighted): 0.9897
F1-Score (weighted): 0.8987
Specificity: 0.6453
AUC: 0.8175


In [92]:
T_and_T(pneumonia_dataset_balanced_224, "microsoft/resnet-50", "resnet_pneumonia_balanced_224", 16, 0.1, True, True, resnet_freezer)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['resnet.encoder.stages.3.layers.2.layer.0.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.0.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.1.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.1.normalization.bias', 'resnet.encoder.stages.3.layers.2.layer.2.convolution.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.weight', 'resnet.encoder.stages.3.layers.2.layer.2.normalization.bias', 'classifier.1.weight', 'classifier.1.bias']


C:\Users\baiet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\baiet\AppData\Local\Temp\ipykernel_4812\2090117231.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_beit = Trainer(


Epoch,Training Loss,Validation Loss
1,0.248100,0.177316
2,0.162700,0.174311
3,0.179400,0.145925
4,0.151000,0.153527
5,0.137600,0.164797


Accuracy: 0.8942
Precision (weighted): 0.8750
Recall (weighted): 0.9692
F1-Score (weighted): 0.9197
Specificity: 0.7692
AUC: 0.8692
